In [4]:
from tinydb import TinyDB, Query
import numpy as np
import os.path
import operator

In [5]:
def inner_trellis(file,m):
    if(os.path.isfile(file)):
        return TinyDB(file)
    else:
        trel = TinyDB(file)
        query = Query()
        if(not (len(trel.search(query.init.exists())) == m*2)):
            os.remove(file)
            trel = TinyDB(file)
            logm = int(np.log2(m))
            for inp in range(m):
                A = ([int(j) for j in format(inp,'#0'+str(logm + 2) + 'b')[2:]])
                for si in range(2):
                    PPM = si
                    for i in range(len(A)):
                        PPM = 2*PPM*(int(i%logm != 0)) + (PPM & 1 ^ A[i])
                        if i%logm == (logm - 1):
                            trel.insert(
                                {"initial":tuple([si]),
                                 "terminal":tuple([PPM&1]),
                                 "input":tuple(A),
                                 "output":tuple([int(j == PPM) for j in range(m)])
                                })
                            PPM = PPM & 1
        return trel

In [6]:
def outer_trellis(file,rate):
    if(os.path.isfile(file)):
        return TinyDB(file)
    else:
        trel = TinyDB(file)
        for si in [[i,j] for i in range(2) for j in range(2)]:
            out = [None,None];
            for inp in range(2):
                out[0] = inp ^ si[0]
                out[1] = inp ^ si[0] ^ si[1]
                trel.insert({"initial":si,
                             "terminal":[inp,si[0]],
                             "input":[inp],
                             "output":out})
        return trel

In [7]:
inner = inner_trellis("inner_trellis.json",16)
outer = outer_trellis("outer_trellis.json",1)

In [8]:
def to_string(x):
    return ''.join([str(int(i)) for i in x])
def ungroup(U):
    a = []
    for i in U:
        for j in i:
            a.append(j)
    return a

In [9]:
def encode(U,trellis):
    query = Query()
    temp = trellis.search(query.initial.exists())[0]
    schema = {"initial": len(temp["initial"]),
             "terminal": len(temp["terminal"]),
             "input":len(temp["input"]),
             "output": len(temp["output"])}
    msglen = len(U)
    initial = [0 for i in range(schema["initial"])]
    output = []
    for i in range(int(msglen/schema["input"])):
        edge = trellis.search(
            (query.initial == initial) & 
            (query.input == U[i*schema["input"]:(i+1)*schema["input"]]))
        assert(len(edge) == 1)
        output.append(edge[0]["output"])
        initial = edge[0]["terminal"]
    return ungroup(output)

In [10]:
def channel(C,Ks,Kb):
    Y = [np.random.poisson(Kb) + np.random.poisson(Ks) if C[i] == 1 
         else np.random.poisson(Kb) for i in range(len(C))]
    return Y

In [11]:
def interleave(X):
    pi = interleaver_map(len(X))
    return [X[i] for i in pi]
def interleaver_map(N):
    pi_map = [None for i in range(N)]
    pi_map[0] = 0
    b = 221
    for i in range(1,N):
        pi_map[i] = (pi_map[i-1] + b)%N
        b = (b+420)%N
    return pi_map

In [12]:
def quantize(n,C=8):
    return -127 if (C*n<-127) else 127 if (C*n>127) else int(C*n)
def many_max_star(L):
    val = L[0]
    for i in range(1,len(L)):
        val = max_star(val,L[i])
    return val
def max_star(A,B):
    return max(A,B) + max_star_LUT(abs(A-B))
def max_star_LUT(i):
    a = { 0:6, 1:5,  2:5,  3:4,  4:4,  5:3,  6:3,  7:3,  8:3,  9:2, 10:2, 
         11:2,12:2, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 20:1, 21:1}
    if i < 22:
        return a[i]
    else:
        return 0

In [13]:
def calc_gamma(Y,LLR,trellis,Ks,Kb):
    query = Query()
    m = len(trellis.all()[0]["output"])
    gamma = [{} for i in range(int(len(Y)/m))]
    logm = int(np.log2(m))
    for i in range(int(len(Y)/m)):
        y = Y[i*m:i*m+m]
        llr = LLR[i*logm:i*logm+logm]
        for edge in trellis.search(query.terminal.exists()):
            gamma[i][edge.doc_id] = quantize(gamma_one(y,Ks,Kb,edge["output"]) + 
                                     gamma_two(llr,edge["input"]))
    return gamma
def gamma_one(Y,Ks,Kb,output):
    c = output.index(max(output))
    return Y[c]*np.log(1+Ks/Kb)
def gamma_two(LLR,A):
    total = 0
    for i in range(len(A)):
        total += LLR[i]*(-1)**(A[i]+1)
    return total

In [14]:
def calc_sgamma(gam,trellis):
    m = len(trellis.all()[0]["output"])
    sgam = [{} for i in range(len(gam))]
    for i in range(len(gam)):
        for edge in gam[i]:
            val = trellis.get(doc_id = edge)
            transition = tuple(val["initial"] + val["terminal"])
            if transition in sgam[i]:
                sgam[i][transition] = max_star(gam[i][edge],sgam[i][transition])
            else:
                sgam[i][transition] = gam[i][edge]
    return sgam

In [112]:
def calc_alpha(sgam,init_alpha,trellis):
    m = len(trellis.all()[0]["output"])
    alph = [{} for i in range(len(sgam))]
    alph[0] = {tuple([i]):init_alpha[i] for i in init_alpha.keys()}
    states = init_alpha.keys()
    for i in range(1,len(sgam)):
        for outstate in states:
            for instate in states:
                if outstate in alph[i]:
                    alph[i][tuple([outstate])] = max_star(alph[i-1][tuple([instate])]+
                                                 sgam[i][tuple([instate]+[outstate])],
                                                 alph[i][tuple([outstate])])
                else:
                    alph[i][tuple([outstate])] = alph[i-1][tuple([instate])]+sgam[i][tuple([instate]+[outstate])]
    return alph

In [108]:
def calc_beta(sgam,fin_beta,trellis):
    m = len(trellis.all()[0]["output"])
    beta = [{} for i in range(len(sgam))]
    beta[-1] = {tuple([i]):fin_beta[i] for i in fin_beta.keys()}
    states = fin_beta.keys()
    for i in range(len(sgam)-1,0,-1):
        for instate in states:
            for outstate in states:
                if instate in beta[i-1]:
                    beta[i-1][tuple([instate])] = max_star(beta[i][tuple([outstate])]+
                                                 sgam[i][tuple([instate] + [outstate])],
                                                 beta[i-1][tuple([instate])])
                else:
                    beta[i-1][tuple([instate])] = beta[i][tuple([outstate])] + sgam[i][tuple([instate] + [outstate])]
    return beta

In [110]:
def calc_sigma(alpha,beta,gamma,trellis):
    sigma = [{} for i in range(len(gamma))]
    for i in range(1,len(gam)):
        for edge in gam[i]:
            val = trellis.get(doc_id = edge)
            sigma[i][edge] = (alpha[i-1][tuple(val["initial"])] + 
                             gamma[i][edge] + 
                             beta[i][tuple(val["terminal"])])
    return sigma

In [70]:
r = [1,2]
m = 16
Ks = 3
Kb = .001
U = [0,1,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0]
X = encode(U,outer_trellis("outer_trellis.json",r))
A = interleave(X)
C = encode(A,inner_trellis("inner_trellis.json",m))
Y = channel(C,Ks,Kb)

In [114]:
query = Query()
LLRAin = [0 for i in range(len(U)*r[1])]
ab_init = {0:0,1:-127}
gam = calc_gamma(Y,LLRAin,inner,Ks,Kb)
sgam = calc_sgamma(gam,inner)
alph = calc_alpha(sgam,ab_init,inner)
beta = calc_beta(sgam,ab_init,inner)
sigma = calc_sigma(alph,beta,gam,inner)
sigma

[{},
 {1: 382,
  2: 255,
  3: 509,
  4: 255,
  5: 382,
  6: 255,
  7: 382,
  8: 255,
  9: 382,
  10: 255,
  11: 382,
  12: 255,
  13: 382,
  14: 255,
  15: 382,
  16: 255,
  17: 382,
  18: 255,
  19: 382,
  20: 382,
  21: 382,
  22: 255,
  23: 382,
  24: 255,
  25: 382,
  26: 255,
  27: 382,
  28: 255,
  29: 382,
  30: 255,
  31: 382,
  32: 255},
 {1: 144,
  2: 255,
  3: 271,
  4: 255,
  5: 144,
  6: 255,
  7: 144,
  8: 255,
  9: 144,
  10: 255,
  11: 144,
  12: 255,
  13: 144,
  14: 255,
  15: 144,
  16: 255,
  17: 144,
  18: 255,
  19: 144,
  20: 382,
  21: 144,
  22: 255,
  23: 144,
  24: 255,
  25: 144,
  26: 255,
  27: 144,
  28: 255,
  29: 144,
  30: 255,
  31: 144,
  32: 255},
 {1: 255,
  2: 366,
  3: 255,
  4: 366,
  5: 255,
  6: 366,
  7: 255,
  8: 366,
  9: 255,
  10: 366,
  11: 255,
  12: 366,
  13: 382,
  14: 366,
  15: 255,
  16: 366,
  17: 255,
  18: 366,
  19: 255,
  20: 366,
  21: 255,
  22: 366,
  23: 255,
  24: 366,
  25: 255,
  26: 366,
  27: 255,
  28: 366,
  29: 25

In [67]:
max_star(+sgam[len(sgam)-1][(1,0)],-127+sgam[len(sgam)-1][(1,1)])

64

In [68]:
sgam

[{(0, 0): 16, (0, 1): 127, (1, 0): 16, (1, 1): 127},
 {(0, 0): 16, (0, 1): 127, (1, 0): 16, (1, 1): 127},
 {(0, 0): 16, (0, 1): 127, (1, 0): 16, (1, 1): 127},
 {(0, 0): 64, (0, 1): 16, (1, 0): 64, (1, 1): 16},
 {(0, 0): 127, (0, 1): 64, (1, 0): 127, (1, 1): 64},
 {(0, 0): 127, (0, 1): 16, (1, 0): 127, (1, 1): 16},
 {(0, 0): 64, (0, 1): 16, (1, 0): 64, (1, 1): 16},
 {(0, 0): 127, (0, 1): 16, (1, 0): 127, (1, 1): 16},
 {(0, 0): 127, (0, 1): 16, (1, 0): 127, (1, 1): 16},
 {(0, 0): 16, (0, 1): 127, (1, 0): 16, (1, 1): 127},
 {(0, 0): 16, (0, 1): 64, (1, 0): 16, (1, 1): 64},
 {(0, 0): 16, (0, 1): 16, (1, 0): 16, (1, 1): 16},
 {(0, 0): 64, (0, 1): 16, (1, 0): 64, (1, 1): 16}]

In [64]:
-127+sgam[len(sgam)-1][(1,1)]

-111

In [74]:
alph

[{0: 0, 1: -127},
 {0: 16, 1: 127},
 {0: 143, 1: 254},
 {0: 381, 1: 270},
 {0: 508, 1: 397},
 {0: 635, 1: 524},
 {0: 762, 1: 651},
 {0: 889, 1: 778},
 {0: 1016, 1: 905},
 {0: 1032, 1: 1143},
 {0: 1159, 1: 1270},
 {0: 1334, 1: 1286},
 {0: 1461, 1: 1350}]